In [1]:
import pandas as pd 
from pathlib import Path
import os
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from haversine import haversine, Unit

from datetime import datetime
from dateutil.parser import parse
from dateutil import tz

def convert(time_string, tzinfos={"ET": tz.gettz("US/Eastern")}):
    """
    convert string in the format [HH]:[MM] AM ET or [HH]:[MM] PM ET to 24-hour time in CT
    """
    date_obj = parse(time_string, tzinfos=tzinfos)
    return  str(date_obj.astimezone(tz=None).strftime('%Y-%m-%d_%H-%M-%S'))


import winsound
frequency = 1760 # A6 on piano
duration = 1000  # Set Duration To 1000 ms == 1 second

In [2]:
output_path = 'output_Chicago'
my_loc = (41.840573846853516, -87.63928590242628)
chromedriver_path = Path('C:/Users/pipih/Documents/Drives/chromedriver.exe')

df = pd.read_csv('data/IL_towns_and_cities.csv')
df = df.drop_duplicates(subset=['name', 'latitude', 'longitude'])
df.set_index('name', inplace=True)

max_distance = 200 # in miles

In [4]:
###### Use Selenium to scrape CVS info timed loop
while True:
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print(f'\nChecking for appointments at {current_time}')

    # Get current appointment availability
    url = 'https://www.cvs.com/immunizations/covid-19-vaccine?icid=cvs-home-hero1-link2-coronavirus-vaccine'
    timestamp_xpath = '//*[@id="vaccineinfo-IL"]/div/div/div/div[1]/div[2]/div/div/div[2]/div/div[5]/div/p[1]'
    data_xpath = '//*[@id="vaccineinfo-IL"]/div/div/div/div[1]/div[2]/div/div/div[2]/div/div[6]/div/div/table/tbody'

    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--log-level=3')

    driver = webdriver.Chrome(options=chrome_options, executable_path=chromedriver_path)
    driver.get(url)
    driver.find_element_by_link_text('Illinois').click()

    try:
        # 
        # timestamp
        timestamp = driver.find_element_by_xpath(timestamp_xpath).text
        timestamp = timestamp.split('.')[0].lstrip('Status as of ')
        print(timestamp)

        timestamp_local = convert(timestamp)
        print(f"\n{timestamp_local}")

        # data
        data = driver.find_element_by_xpath(data_xpath).text
        driver.quit()
        cvs_locs = [line.split(', IL ') for line in data.split('\n')]
        cvs_locs = [loc for loc in cvs_locs if loc[-1] == 'Available']


        # get open locations with GPS coords
        locations = []
        for loc in cvs_locs:
            name = loc[0]        
            if name in df.index:
                lat, lon = list(map(float, df.loc[name, ['latitude', 'longitude']]))
                cvs_loc = (lat, lon)
                distance = haversine(my_loc, cvs_loc, unit='mi')
                locations.append([name, lat, lon, distance])
        
        locations = pd.DataFrame(data=locations, columns=['name', 'latitude', 'longitude', 'distance'])
        locations = locations[locations.distance < max_distance].sort_values(by='distance').set_index('name')
        print(f'Found {len(locations)} CVS locations with appointments available:')
        print(locations)

        if len(locations) > 0:
            winsound.Beep(frequency, duration)
            output_filename = f'{output_path}/appointments_found_{timestamp_local}.txt'
            locations.to_csv(output_filename)
    
    except:
        print('something might be wrong')

    # check every 15 minutes (CVS official info refreshes at this rate)
    print('\nChecking again in 15 minutes...')
    time.sleep(900)


Checking for appointments at 12:23:04
1:17 PM ET

2021-04-05_12-17-00
Found 4 CVS locations with appointments available:
             latitude  longitude    distance
name                                        
Rock Island   41.3034   -90.3443  144.658087
Canton        40.3329   -90.0206  161.945563
Decatur       39.5025   -88.5717  168.770337
Springfield   39.4806   -89.3837  186.931858

Checking again in 15 minutes...

Checking for appointments at 12:38:12
1:32 PM ET

2021-04-05_12-32-00
Found 4 CVS locations with appointments available:
             latitude  longitude    distance
name                                        
Rock Island   41.3034   -90.3443  144.658087
Canton        40.3329   -90.0206  161.945563
Decatur       39.5025   -88.5717  168.770337
Springfield   39.4806   -89.3837  186.931858

Checking again in 15 minutes...

Checking for appointments at 12:53:23
1:47 PM ET

2021-04-05_12-47-00
Found 4 CVS locations with appointments available:
             latitude  longi

KeyboardInterrupt: 